In [1]:
""" 
Created on Wed Jan 23 2019
 
MTS Program that graphs difference in mahalanobis distances, as well as an MTS 
 
@author: Taehee 
""" 
#
from __future__ import print_function
import sys
import os
os.chdir("C:/Users/mando/Desktop/MTS_system/base_data")

# DB연결

import os
import pyodbc  
# 수치계산
import numpy as np 
import matplotlib as plt 
import copy 
import math
import csv
import pandas as pd
from sklearn import preprocessing

#--------------------------------------------------------------------------------#

# OKdata_factor_mean1,2
# OKdata_factor_std 1,2를 각각 받아오거나 하드코딩으로 저장해야 한다.

In [2]:
# mssql 접속 2개 만들기 
conn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=MDPLP1546701\\SQLEXPRESS;"
                      "Database=ROLLCURLING_MONITORING;"
                      "Trusted_Connection=yes;")

#connection에서 cursor 생 (cursor는 fetch 동작 관리)
cursor = conn.cursor()
# OS와 맞는 DB설치할 것

In [3]:
cursor.execute("SELECT * from dbo.QMS_MONITORING_DATA  WHERE TIME = (select MAX(TIME) FROM dbo.QMS_MONITORING_DATA where DATE = (select max(DATE) FROM dbo.QMS_MONITORING_DATA where FLAG='Y') AND FLAG='Y') AND FLAG='Y' ORDER BY DATE DESC ; ")
    # from dbo.QMS_MONITORING_DATA  WHERE TIME = (select MAX(TIME) FROM dbo.QMS_MONITORING_DATA where DATE = (select max(DATE) FROM dbo.QMS_MONITORING_DATA where FLAG='Y') AND FLAG='Y') AND FLAG='Y' ORDER BY DATE DESC                          
    # VALUE01 : 축력하중(kgf)
    # VALUE02 : 컬링하중(kgf)
    # VALUE03 : 토크상한 누적시간
    # VALUE04 : RPM 편차(rpm)
    # VALUE05 : 높이측정(mm) (사용x)
    # VALUE06 : 완료위치(mm) (사용x)
    # VALUE07 : 높이측정-완료위치(mm)
    # VALUE08 : 단차 측정(mm) (SAM2라인 사용X)
    
row =cursor.fetchone()
print(row)

# base data에 넣기 위한 할당
date=row[0]
time=row[1]
wc=row[2]
car_type=row[4]
serial_n=row[5]
var_aweight=row[13]
var_cweight=row[14]
var_tqu=row[15]
var_rpm=row[16]
var10=None
var_h=row[19]
var13=None
var14=None

('2019-01-02', '15:22:27', 'SAM2', '190102', '116', '963', Decimal('7.00000'), Decimal('15.00000'), Decimal('0.74918'), None, None, 'Y', '0', Decimal('1718.00000'), Decimal('2177.00000'), Decimal('0.03200'), Decimal('48.00000'), Decimal('89.90000'), Decimal('61.60000'), Decimal('28.30000'), None, None, None)


In [16]:
if row[2] == 'SAM2': # 차종 
    if row[4]== '140' or row[4] =='141' or row[4] == '142':
        print('1')
        # 여기서 각 모델을 돌림
        # db을 쌓음
    else:
        print('2')
else: # SAA8인경우 
    if row[2]== '10' or row[2] =='12':
        
        print('3')
    else:
        print('4')

2


In [6]:
### 기능 3번 

cursor.execute( "insert into dbo.MTS_BASE_DATA (WC,DATE,TIME,CAR_TYPE,SERIAL_NO,VAR_AWEIGHT,VAR_CWEIGHT,VAR_TQU_CTIME,VAR_RPM_BIAS,VAR_H_MINUS_CL) VALUES ('%s','%s','%s','%s','%s','%s','%s','%s','%s','%s');"%(wc,date,time,car_type,serial_n,var_aweight,var_cweight,var_tqu,var_rpm,var_h)     )
cursor.commit()

IntegrityError: ('23000', "[23000] [Microsoft][SQL Server Native Client 11.0][SQL Server]Violation of PRIMARY KEY constraint 'PK_MTS_BASE_DATA_1'. Cannot insert duplicate key in object 'dbo.MTS_BASE_DATA'. The duplicate key value is (SAM2, 2019-01-02, 116, 963, 15:22:27). (2627) (SQLExecDirectW); [23000] [Microsoft][SQL Server Native Client 11.0][SQL Server]The statement has been terminated. (3621)")

In [22]:
#### 기능 2번 


# DB data 불러오기
row_array = []
z_array = []

cursor.execute("SELECT VALUE01,VALUE02,VALUE03,VALUE04,VALUE07 from dbo.QMS_MONITORING_DATA  WHERE TIME = (select MAX(TIME) FROM dbo.QMS_MONITORING_DATA where DATE = (select max(DATE) FROM dbo.QMS_MONITORING_DATA where FLAG='Y') AND FLAG='Y') AND FLAG='Y' ORDER BY DATE DESC ; ")

row = cursor.fetchone()
print("raw data:", row)
row_array = np.array(row)
k=5

# base data 불러오기
OKdata_factor_mean=pd.read_csv("C:/Users/mando/Desktop/MTS_system/base_data/OKdata_factor_mean.csv")
OKdata_factor_std=pd.read_csv("C:/Users/mando/Desktop/MTS_system/base_data/OKdata_factor_std.csv")
cor=pd.read_csv("C:/Users/mando/Desktop/MTS_system/base_data/cor.csv")

OKdata_factor_mean=np.array(OKdata_factor_mean.drop(['Unnamed: 0'],1))
OKdata_factor_std=np.array(OKdata_factor_std.drop(['Unnamed: 0'],1))
cor=np.array(cor.drop(['Unnamed: 0'],1))


#MD 연산

for i in range(k):

    data_minus_mean = float(row_array[i])-OKdata_factor_mean[0][i]
    z = data_minus_mean/OKdata_factor_std[0][i]
    #print("z:",z)
    z_array.append(z)

print("z:", z_array)
z_transpose = np.transpose(z_array)
#print(z_transpose)
MD_1 = np.dot(z_array,cor)
MD = np.dot(MD_1, z_transpose)
D = np.sqrt(MD)

print("MD:",MD,",","D:",D)

raw data: (Decimal('1718.00000'), Decimal('2177.00000'), Decimal('0.03200'), Decimal('48.00000'), Decimal('28.30000'))
z: [0.7922601254935119, 0.5847899171797744, -1.09343469358407, 0.922769028474672, 1.1776725642714965]
MD: 2.2390249158906115 , D: 1.4963371665138214
